**Prod Version of DevEnv**

**This notebook is run using nbconvert after devenv service starts**

In [1]:
!pip install twitter
!pip install feedparser
!pip install schedule
!pip install kafka-python
!pip install redis
!pip install uuid

In [13]:
from time import sleep
from json import dumps, loads, load
from kafka import KafkaProducer
from datetime import datetime, timedelta
from feedparser import parse
from urllib.request import urlopen
from twitter import Twitter
import schedule
from datetime import datetime
import uuid
from pytz import timezone
from bs4 import BeautifulSoup
import urllib.request 
import pandas as pd

In [44]:
tickers = ["MSFT","AAPL"]
feeds = ["http://feeds.reuters.com/reuters/businessNews",
         "https://finance.yahoo.com/news/rssindex/", 
         "https://www.nasdaq.com/feed/rssoutbound",
         "https://news.microsoft.com/feed",
        "https://feeds.a.dj.com/rss/WSJcomUSBusiness.xml",
        "https://feeds.a.dj.com/rss/RSSMarketsMain.xml",
        "http://www.marketwatch.com/rss/topstories",
        "http://www.marketwatch.com/rss/marketpulse",
        "http://www.marketwatch.com/rss/realtimeheadlines",
        "http://feeds.marketwatch.com/marketwatch/bulletins",
        "https://www.cnbc.com/id/10000664/device/rss/rss.html",
        "https://www.cnbc.com/id/10001147/device/rss/rss.html",
        "http://feeds.feedburner.com/TechCrunch/",
        "https://www.ft.com/?format=rss",
        "https://rss.nytimes.com/services/xml/rss/nyt/Business.xml",
        "https://abcnews.go.com/abcnews/moneyheadlines",
        "https://www.cbsnews.com/latest/rss/main",
        "https://markets.businessinsider.com/rss/news",
        "http://newsrss.bbc.co.uk/rss/newsonline_uk_edition/front_page/rss.xml",
        "http://feeds.bbci.co.uk/news/business/rss.xml"]

bootstrapping = True
last_time_step = datetime.now(tz=timezone('US/Pacific'))-timedelta(days=3)
output = {}

producer = KafkaProducer(
    bootstrap_servers=['broker:9092'],
    api_version= (0,11),
    value_serializer=lambda x: dumps(x).encode('utf-8'))

# Authenticate with Twitter & search
# https://pypi.org/project/twitter/
# https://developer.twitter.com/en/apps/17019807
from twitter import *
t = Twitter(auth=OAuth(consumer_key="nDwKHutvaskyGecZkCS1SiYY8", consumer_secret="zlQdJIuFUGGglbMO2kYMk6oNLFztzipxhRANNz5q5gqfn1tGb1", token="73568030-GqfoqkfWRhqhARS9uzz51vg1UGSV7AGRaOxDkAurZ", token_secret="1kiHGfyDfO7gcaJbGFnr3lnLA3vXc8ti9hJhj1lnTTvEm"))

In [54]:
def DownloadStreams():
    # Yahoo Finance Stream for Stock Price
    f_data = {}
    for ticker in tickers:
        try:
            y_finance = load(urlopen("https://query1.finance.yahoo.com/v7/finance/chart/"+ticker+"?range=1h&interval=1h"))
            meta = y_finance['chart']['result'][0]['meta']
            timestamp = meta['regularMarketTime']
            price = meta['regularMarketPrice']
            f_data[ticker] = {"Type": "Finance", "Ticker": ticker, "Timestamp":timestamp, "Price":price}
        except:
            print("Exception in Download Stream for "+ticker)
    
    # News Stream from Reuters for Business
    try:
        if(None == last_time_step):
            last_time_step = datetime.now(tz=timezone('US/Pacific'))-timedelta(days=3)
    except NameError:
        last_time_step = datetime.now(tz=timezone('US/Pacific'))-timedelta(days=3)
        
    feed_data = {}
    for feed in feeds:
        try:
            feed_data[feed] = [{"Type": "News", "Timestamp": datetime.timestamp(pd.to_datetime(l.published)), "Title":l.title, "Summary":BeautifulSoup(l.summary).get_text(strip=True), "Link":l.link} for l in (parse(feed))["entries"] if (pd.to_datetime(l.published)>last_time_step and l.title not in output and len(l.summary)<500)]
        except TypeError:
            print("TypeError")
        # print([l.title for l in d["entries"]])
    
#    if(bootstrapping):
        
    
    last_time_step = datetime.now(tz=timezone('US/Pacific'))-timedelta(days=3)
    for feed in feeds:
        for entry in feed_data[feed]:
            output[entry['Title']]=datetime.fromtimestamp(int(entry['Timestamp']),tz=timezone('US/Pacific'))
        
    deletion_keys = [val for val in output if(output[val]<last_time_step)]
    for key in deletion_keys:
        output.pop(key,None)
    
    return f_data, feed_data

def UploadStreamsToKafka(f_data, feed_data, PopularityOnly=False):
    if(not PopularityOnly):
        for key in f_data.keys():
            producer.send('finance_ticker', value=f_data[key])
        for key in feed_data.keys():
            for entry in feed_data[key]:
                producer.send('news_feed', value=entry)
    # Create Popularity Click Feed
#    for key in feed_data.keys():
#        for entry in feed_data[key]:
#            a = t.search.tweets(q=entry["Title"])
#            for s in a['statuses']:
#                if(s['retweet_count']>0):
#                    producer.send('click',value={'Type': 'Click', 'User': 'generic_'+str(uuid.uuid4()), 'Timestamp': datetime.timestamp(datetime.now()), 'Title': entry["Title"], 'Link': entry["Link"] })
    producer.flush()

def HourlyJob():
    # Check, Download & Parse each stream
    f_data, feed_data = DownloadStreams()
    # Upload cleaned events into Kafka Topics
    UploadStreamsToKafka(f_data, feed_data)
    print("Hourly Run Events Complete - Sleeping now")

def MinutelyJob():
    f_data, feed_data = DownloadStreams()
    UploadStreamsToKafka(f_data, feed_data, True)
    print("Minutely Run Events Complete - Sleeping now")



In [55]:
HourlyJob()

Hourly Run Events Complete - Sleeping now


In [7]:
MinutelyJob()

Minutely Run Events Complete - Sleeping now


In [ ]:
schedule.clear()
HourlyJob()

schedule.every(15).minutes.do(HourlyJob)
schedule.every(10).minutes.do(MinutelyJob)
starttime = datetime.now()
endtime = starttime + timedelta(minutes=5)

while True: #datetime.now()<endtime:
    schedule.run_pending()
    sleep(10)
    
schedule.clear()

In [47]:
output

{'Tesco suspends Chinese supplier after prisoner labor report': datetime.datetime(2019, 12, 22, 2, 17, 17, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
 'The best year financial markets have ever had?': datetime.datetime(2019, 12, 21, 22, 50, 2, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
 "'Not for the faint of heart:' Critical landing test ahead for Boeing Starliner": datetime.datetime(2019, 12, 21, 22, 28, 43, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
 "Trump says trade deal with China to be signed 'very shortly'": datetime.datetime(2019, 12, 21, 22, 11, 54, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
 "Manager of McDonald's franchise in Peru says faulty drinks machine caused deaths": datetime.datetime(2019, 12, 21, 17, 28, 12, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
 'NASA, Boeing investigating why Starliner spacecraft missed orbit': datetime.datetime(2019, 12, 21, 13, 18, tzinfo=<DstTzInfo 'US/Pacific' PST-